In [ ]:
import numpy as np
import os
import matplotlib as plt
from PIL import Image
import PIL
from tqdm import tqdm
import glob 
import csv
dataset_dir ='/home/ioannis/Thesis/Datasets/ChestXR/CheXpert-v1.0-small/'
train_dir = '/home/ioannis/Thesis/Datasets/ChestXR/CheXpert-v1.0-small/train/patient*'
val_dir = '/home/ioannis/Thesis/Datasets/ChestXR/CheXpert-v1.0-small/valid/patient*'
plt.rcParams['figure.dpi'] = 100

In [ ]:
print ('Trainset:',len(glob.glob(train_dir)),'patients')
print ('Validation Set:',len(glob.glob(val_dir)),'patients')

In [ ]:
print ('Trainset:',len(glob.glob(train_dir+'/study*/view*.jpg')),'patients')
print ('Validation Set:',len(glob.glob(val_dir)),'patients')

Open CSV. It contains all sample info as path_to_jpg|sex|age|frontal_or_lateral|class scores.  The first class is "No finding" indicating healthy sample. Positive label is 1. We will write 2 csvs that will contain the links of positives, one for each sex only for frontal (lateral has much lower number of samples so discard). We will keep study2 if it is done since the studies seem to have different appearences and would be worthwile to keep for a richer dataset. 


In [ ]:
csv_keys = ['Path', 'Sex', 'Age', 'Frontal/Lateral', 'AP/PA', 'No Finding', 
                'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion', 
                'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices']

Chest Posterior Anterior (PA) and Anterior Posterior (AP) radiographs. Should we take only one or the other? 
It would make sense to take both if they are of same left-right orientation. consider class imballance though --> seems balanced ~5k each 

In [ ]:
with open(dataset_dir + 'train.csv', newline='') as csvfile:
    with open(dataset_dir + 'female_anomal_effusion_AP_no_sup_3419.txt', 'w', newline='') as female, open(dataset_dir + 'male_anomal_effusion_AP_no_sup_4036.txt', 'w', newline='') as male:

        iterator = csv.DictReader(csvfile, delimiter=',', quotechar='|')    
        count_m_f = [0,0]

        for row in iterator:
            
            if row['Enlarged Cardiomediastinum'] != '1.0' and row['Frontal/Lateral'] == 'Frontal' and row['Age']>'18' and  row['AP/PA'] == 'PA' :
                if row['Lung Opacity'] != '1.0' and row['Fracture'] != '1.0' and row['Pleural Other'] != '1.0' and  row['Pleural Effusion'] == '1.0' and row['Pneumothorax'] != '1.0':
                    if row['Support Devices'] != '1.0':
                        if row['Sex'] == 'Male':
                            male.write(row['Path'] + "\n")
                            count_m_f[0] += 1

                        elif row['Sex'] == 'Female':
                            female.write(row['Path'] + "\n")
                            count_m_f[1] += 1   
        
count_m_f    

In [ ]:
with open(dataset_dir + 'train.csv', newline='') as csvfile:
    with open(dataset_dir + 'female_normal_train_AP_sup_5264.txt', 'w', newline='') as female, open(dataset_dir + 'male_normal_train_AP_sup_6134.txt', 'w', newline='') as male:

        iterator = csv.DictReader(csvfile, delimiter=',', quotechar='|')    
        count_m_f = [0,0]

        for row in iterator:
            
            if row['No Finding'] == '1.0' and row['Frontal/Lateral'] == 'Frontal' and row['AP/PA'] == 'AP'  and row['Age']>'18':

                if row['Sex'] == 'Male':
                    male.write(row['Path'] + "\n")
                    count_m_f[0] += 1

                elif row['Sex'] == 'Female':
                    female.write(row['Path'] + "\n")
                    count_m_f[1] += 1   
      
count_m_f    

In [ ]:
data_dir ='/home/ioannis/Thesis/Datasets/ChestXR/'
with  open(dataset_dir + 'male_normal_train.txt', 'r', newline='') as male:
    paths = male.read().splitlines()
    #print(paths)
    for i in range(20):
        
        print (np.asarray(Image.open(data_dir+paths[i])).shape)



some normal patients exist in valid set, 26 samples, just ignore those. Valid set is hand labeled instead of AI labeled.

**Chexplanation preproc:**

In [ ]:
from pycocotools import mask
import matplotlib.pyplot as plt
import json

with open('/datasets/Datasets/ChestXR/chexplanation_dataset/gt_segmentation_val.json') as json_file: 
    GT_data = json.load(json_file)
 
#GT_data is a dict with keys of type 'patient64622_study1_view1_frontal'-->
# (['Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Lesion', 'Airspace Opacity', 'Edema', 'Consolidation', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Support Devices']) -->
# -->['size', 'counts'] which is a subdictionary that can be decoded by the mask.decode(x) function and output the binary segm image


In [ ]:
from pycocotools import mask
import matplotlib.pyplot as plt
import json

with open('/datasets/Datasets/ChestXR/chexplanation_dataset/gt_segmentation_val.json') as json_file: 
    GT_data = json.load(json_file)
 
patient_list = list(GT_data.keys())

for idx, patient in enumerate(patient_list):
    if patient.split('_')[-1] == 'lateral':
        patient_list.pop(idx)

pathology_list = ['Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Lesion', 'Airspace Opacity', 'Edema', 'Consolidation', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Support Devices']
import numpy as np
from PIL import Image
for i in patient_list:
    #if 'Pleural Effusion' in GT_data[i].keys():
    pathol_list = []
    gt = []
    paths = []
    for j in range(len(pathology_list)):
        img = mask.decode(GT_data[i][pathology_list[j]])
        if not (img == np.zeros_like(img)).all():
            pathol_list.append(pathology_list[j])

    if len(pathol_list) == 1 and pathol_list[0] == 'Pleural Effusion'  :
        gt.append(img)
        patient_num, study, fr_lat = i.split("_", 2)
        path2 = '/datasets/Datasets/ChestXR/CheXpert-v1.0-small/valid/'+ patient_num +'/' + study+'/' + fr_lat+ '.jpg'
        paths.append(path2)
      

In [ ]:
pathology_list = ['Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Lesion', 'Airspace Opacity', 'Edema', 'Consolidation', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Support Devices']
import numpy as np
from PIL import Image
for i in patient_list:
    #if 'Pleural Effusion' in GT_data[i].keys():
    pathol_list = []
    gt = []
    paths = []
    for j in range(len(pathology_list)):
        img = mask.decode(GT_data[i][pathology_list[j]])
        if not (img == np.zeros_like(img)).all():
            pathol_list.append(pathology_list[j])

    if len(pathol_list) == 1 and pathol_list[0] == 'Pleural Effusion'  :
        gt.append(img)
        patient_num, study, fr_lat = i.split("_", 2)
        path2 = '/datasets/Datasets/ChestXR/CheXpert-v1.0-small/valid/'+ patient_num +'/' + study+'/' + fr_lat+ '.jpg'
        paths.append(path2)
      

In [ ]:
with open(dataset_dir + 'valid.csv', newline='') as csvfile:
        pa_m = 0
        ap_m = 0
        pa_f = 0
        ap_f = 0
        
        iterator = csv.DictReader(csvfile, delimiter=',', quotechar='|')    
        for row in iterator:
            for pat in patient_list:
                if pat.split('_')[0] in row['Path'].split('/') and 'study1' in row['Path'].split('/') and 'view1' in row['Path'].split('/')[-1].split('_'):
                   if row['Frontal/Lateral'] == 'Frontal' and row['No Finding'] == '0.0' and row['Enlarged Cardiomediastinum'] != '1.0':
                        if row['Sex'] == 'Female' and row['AP/PA'] == 'PA':
                            pa_f+=1
                        if row['Sex'] == 'Male' and row['AP/PA'] == 'PA':
                            pa_m +=1
                        if row['Sex'] == 'Female' and row['AP/PA'] == 'AP':
                            ap_f +=1
                        if row['Sex'] == 'Male' and row['AP/PA'] == 'AP':          
                            ap_m +=1  
print (pa_m, pa_f, ap_m, ap_f)       

#chexplanation only has view1, study1 of no finding == '0.0' and some samples that have 0.0 to everything that the bellow code doesnt produce mask for

In [ ]:
len(glob.glob('/home/ioannis/Thesis/Datasets/ChestXR/CheXpert-v1.0-small/valid/patient*/*/*.jpg'))

In [ ]:
def show_pathologies (patient_data, patient_path):

    patient_num, study, fr_lat = patient_path.split("_", 2)
    path = '/home/ioannis/Thesis/Datasets/ChestXR/chexplanation_dataset/cxr_valid/'+ patient_num +'/' + study+'/' + fr_lat+ '.jpg'
    path2 = 'CheXpert-v1.0-small/valid/'+ patient_num +'/' + study+'/' + fr_lat+ '.jpg'
    with open(dataset_dir + 'valid.csv', newline='') as csvfile:
        iterator = csv.DictReader(csvfile, delimiter=',', quotechar='|')    
        for row in iterator:
            if row['Path'] == path2:
                print("Conditions in sample:", [i for i in csv_keys if row[i] == '1.0'], ", sex:", row['Sex'])
                    

    pathology_list = ['Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Lesion', 'Airspace Opacity', 'Edema', 'Consolidation', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Support Devices']
    pathol_list = []
    count_pathologies = 0
    for i in range(len(pathology_list)):
        img = mask.decode(patient_data[pathology_list[i]])
        if not (img == np.zeros_like(img)).all():
            count_pathologies += 1
            pathol_list.append(pathology_list[i])

    plt.figure(figsize=(20,30))

    for i in range(count_pathologies):
        
        img = mask.decode(patient_data[pathol_list[i]])
        img = Image.fromarray(img)
        img = img.resize((256,256), PIL.Image.LANCZOS)
        img = np.asarray(img)
        plt.subplot(1,count_pathologies+1,i+1)
        plt.title(pathol_list[i])
        plt.imshow(img, cmap='gray')
        plt.axis('off')
    print(img.shape)
    #plot real x-ray
   
    real_img = np.asarray(Image.open(path).resize((256,256), PIL.Image.LANCZOS))
    plt.subplot(1,count_pathologies+1,count_pathologies+1)    
    plt.title("CXR")
    plt.imshow(real_img, cmap='gray')
    plt.axis('off')
    print(real_img.shape)
    #plt.tight_layout()  
      
    

def total_segmentation (patient_data):
    pathology_list = ['Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Lesion', 'Airspace Opacity', 'Edema', 'Consolidation', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion','Support Devices'] 

    for i in range(len(pathology_list)):
        
        img = mask.decode(patient_data[pathology_list[i]])
        if (i == 0) :
            total = np.zeros_like(img)
        total += img
    total [total != 0] = 1
    return total

In [ ]:
def save_segmentation_images (patient_data, patient_path):

    patient_num, study, fr_lat = patient_path.split("_", 2)
    path = '/home/ioannis/Thesis/Datasets/ChestXR/chexplanation_dataset/cxr_valid/'+ patient_num +'/' + study+'/' + fr_lat+ '.jpg'
    path2 = 'CheXpert-v1.0-small/valid/'+ patient_num +'/' + study+'/' + fr_lat+ '.jpg'
    save_path = '/home/ioannis/Thesis/Datasets/ChestXR/chexplanation_dataset/cxr_valid/'+ patient_num +'/masks/' + fr_lat+ '_mask_'

    if not os.path.exists('/home/ioannis/Thesis/Datasets/ChestXR/chexplanation_dataset/cxr_valid/'+ patient_num +'/masks/'):
        os.makedirs('/home/ioannis/Thesis/Datasets/ChestXR/chexplanation_dataset/cxr_valid/'+ patient_num +'/masks/')
    


    with open(dataset_dir + 'valid.csv', newline='') as csvfile:
        iterator = csv.DictReader(csvfile, delimiter=',', quotechar='|')    
        for row in iterator:
            if row['Path'] == path2:
                pass
                #print("Conditions in sample:", [i for i in csv_keys if row[i] == '1.0'], ", sex:", row['Sex'])
                    

    pathology_list = ['Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Lesion', 'Airspace Opacity', 'Edema', 'Consolidation', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Support Devices']

    segmented_pathology_list = []

    for pathology in pathology_list:
        img = mask.decode(patient_data[pathology])
        if not (img == np.zeros_like(img)).all():
            segmented_pathology_list.append(pathology)

    for seg in segmented_pathology_list:
        
        img = mask.decode(patient_data[seg])*255
        img = Image.fromarray(img)
        img.save(save_path+seg+'.png')
        #print(save_path)

In [ ]:
for i in patient_list:
    
    save_segmentation_images(GT_data[i], i)


In [ ]:
pathology_list = ['Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Lesion', #pathologies that segmentations exist for
'Airspace Opacity', 'Edema', 'Consolidation', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion']

with open(dataset_dir + 'valid.csv', newline='') as csvfile, open(dataset_dir + 'female_frontal_abnormal_validset.csv', 'w', newline='') as csvfile_female, open(dataset_dir + 'male_frontal_abnormal_validset.csv', 'w', newline='') as csvfile_male:

    iterator = csv.DictReader(csvfile, delimiter=',', quotechar='|')    
    count_m_f = [0,0]


    writer_m = csv.writer(csvfile_male, delimiter=',',
                        quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer_f = csv.writer(csvfile_female, delimiter=',',
                        quotechar='|', quoting=csv.QUOTE_MINIMAL)  

    for row in iterator:
       
        if row['No Finding'] != '1.0'  and row['Frontal/Lateral'] == 'Frontal' and row['Sex'] == 'Male' and row['Age'] > '18':
            if row['Path'].split('/')[2] != 'patient64581' and row['Path'].split('/')[2] != 'patient64629':
                writer_m.writerow([row['Path'],'chexplanation_dataset/cxr_valid/'+row['Path'].split('/')[2]+'/masks/', '1'])

                count_m_f[0] += 1
            
        if row['No Finding'] != '1.0' and row['Frontal/Lateral'] == 'Frontal' and row['Sex'] == 'Female'and row['Age'] > '18': 
            if row['Path'].split('/')[2] != 'patient64581':
                writer_f.writerow([row['Path'],'chexplanation_dataset/cxr_valid/'+row['Path'].split('/')[2]+'/masks/','1'])
           
            count_m_f[1] += 1   
        if row['No Finding'] == '1.0' and row['Support Devices'] != '1.0' and row['Frontal/Lateral'] == 'Frontal' and row['Sex'] == 'Male' and row['Age'] > '18':
            
            writer_m.writerow([row['Path'], 'chexplanation_dataset/cxr_valid/'+row['Path'].split('/')[2]+'/masks/', '0'])
      
            
           
            
        if row['No Finding'] == '1.0' and row['Support Devices'] != '1.0' and row['Frontal/Lateral'] == 'Frontal' and row['Sex'] == 'Female'and row['Age'] > '18': 
            writer_f.writerow([row['Path'], 'chexplanation_dataset/cxr_valid/'+row['Path'].split('/')[2]+'/masks/','0'] )
           
            
        
print(count_m_f)
#print( abnormal_samples, len(abnormal_samples))


In [ ]:
pathology_list = ['Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Lesion', #pathologies that segmentations exist for
'Airspace Opacity', 'Edema', 'Consolidation', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion']

with open(dataset_dir + 'valid.csv', newline='') as csvfile, open(dataset_dir + 'female_frontal_abnormal_validset.csv', 'w', newline='') as csvfile_female, open(dataset_dir + 'male_frontal_abnormal_validset.csv', 'w', newline='') as csvfile_male:

    iterator = csv.DictReader(csvfile, delimiter=',', quotechar='|')    
    count_m_f = [0,0]


    writer_m = csv.writer(csvfile_male, delimiter=',',
                        quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer_f = csv.writer(csvfile_female, delimiter=',',
                        quotechar='|', quoting=csv.QUOTE_MINIMAL)  

    for row in iterator:
       
        if row['Path'].split('/')[2] == 'patient64629':
            print (row)
            

           
            
        
print(count_m_f)
#print( abnormal_samples, len(abnormal_samples))


In [ ]:
csv_keys = ['Path', 'Sex', 'Age', 'Frontal/Lateral', 'AP/PA', 'No Finding', 
                'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion', 
                'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion',
                 'Pleural Other', 'Fracture', 'Support Devices']

pathology_list = ['Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Lesion', #pathologies that segmentations exist for
'Airspace Opacity', 'Edema', 'Consolidation', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion']
dataset_dir ='/home/ioannis/Thesis/Datasets/'
with open(os.path.join(dataset_dir,'ChestXR/CheXpert-v1.0-small/') + 'train.csv', newline='', mode='r') as csvfile:

    iterator = csv.DictReader(csvfile, delimiter=',', quotechar='|')    

    count = 0
   
    for row in iterator:
       
        if row['Enlarged Cardiomediastinum'] != '1.0' and row['Frontal/Lateral'] == 'Frontal' and row['Sex'] == 'Female':
            if row['Age']>'18' and row['No Finding'] == '1.0' :
                count += 1
             
            
                
 #9624 ,7238      
count

these ground truths where produced with the context of saliency--> the doctors are annotating the region of interest ROI corresponding to the lession at hand (similart to LAG datgaset), and according to 20 page of paper these have huge inter expert variance and depends of context of different countries on how to define these anomalies

it should not be used for the problem of pixel level detection but still trying it would be intresting.


pneumonothorax dataset: I cant find anywhere detailed info about the dataset. I dont know if the diagnosis is detailed and no other lessions exist in the images, healthy or pneumono positive. Cant trust healthy or positive samples. As seem in Chexpert dataset, pneumonothorax segmented samples  come with other lessions so the per pixel 
evaluation on pneumonothorax masks will not be accurate. But we can try evaluating nevertheless and see results.




https://www.kaggle.com/jesperdramsch/intro-chest-xray-dicom-viz-u-nets-full-data
https://www.kaggle.com/juliaelliott/siim-reference-model
https://www.kaggle.com/iafoss/data-repack-and-image-statistics
https://www.kaggle.com/onealbao/dicom-to-jpeg-conversion-kernel
https://www.kaggle.com/seesee/siim-train-test
https://www.kaggle.com/retyidoro/eda-of-pneumothorax-dataset
https://www.kaggle.com/maxwell110/dive-into-dicom-datasets
https://www.kaggle.com/abhishek/move-all-dicom-images-to-train-and-test-folders

ony why stanford set is better than chestxray14

https://lukeoakdenrayner.wordpress.com/2017/11/18/quick-thoughts-on-chestxray14-performance-claims-and-clinical-tasks/
https://lukeoakdenrayner.wordpress.com/2017/12/18/the-chestxray14-dataset-problems/
https://lukeoakdenrayner.wordpress.com/2019/02/25/half-a-million-x-rays-first-impressions-of-the-stanford-and-mit-chest-x-ray-datasets/